# L-Systems (Lindenmayer Systems) Lab

## Introduction

L-Systems are parallel rewriting systems that were introduced by Aristid Lindenmayer in 1968. They are particularly useful for modeling plant growth and generating fractals. In this lab, we'll explore how to create various patterns using L-Systems.

### Key Concepts:
- **Axiom**: The initial state/string
- **Production Rules**: Rules that define how to replace characters
- **Iterations**: Number of times to apply the rules
- **Turtle Graphics**: System for visualizing the L-System output

Let's start by importing our required libraries:

In [8]:
! pip install ColabTurtle

In [9]:
import ColabTurtle.Turtle as t
from IPython.display import clear_output
import math
import matplotlib.pyplot as plt

## Part 1: Basic L-System Implementation

First, let's implement our core L-System functions. These will be used throughout the lab to generate and draw various patterns.

In [10]:
def create_l_system(iterations, axiom, rules):
    """Generate L-System instructions based on axiom and rules."""
    result = axiom
    for _ in range(iterations):
        new_string = ""
        for char in result:
            new_string += rules.get(char, char)
        result = new_string
    return result

def draw_l_system(instructions, angle, distance):
    """Draw the L-System using turtle graphics.

    Parameters:
    - instructions: string of L-System commands
    - angle: turning angle in degrees
    - distance: forward movement distance
    """
    stack = []
    for cmd in instructions:
        if cmd == 'F':  # Move forward and draw
            t.forward(distance)
        elif cmd == 'f':  # Move forward without drawing
            t.penup()
            t.forward(distance)
            t.pendown()
        elif cmd == '+':  # Turn right
            t.right(angle)
        elif cmd == '-':  # Turn left
            t.left(angle)
        elif cmd == '[':  # Save current state
            stack.append((t.position(), t.heading()))
        elif cmd == ']':  # Restore previous state
            position, heading = stack.pop()
            t.penup()
            t.goto(position)
            t.setheading(heading)
            t.pendown()

def setup_turtle():
        t.initializeTurtle()
        t.hideturtle()
        t.speed(13)  # Fastest speed
        t.penup()
        t.goto(t.window_width() // 2, t.window_height() - 50)  # Start position
        t.pendown()


## Exercise 1: Koch Curve

The Koch curve is a classic example of a fractal pattern. Let's create it using our L-System:

In [11]:
# Koch curve parameters
koch_axiom = "F"
koch_rules = {"F": "F+F-F-F+F"}
koch_iterations = 3
koch_angle = 90

# Generate and draw
setup_turtle()
t.pensize(2)
koch_instructions = create_l_system(koch_iterations, koch_axiom, koch_rules)
draw_l_system(koch_instructions, koch_angle, 5)

### Exercise 1 Tasks:
1. Try modifying the number of iterations (start with small numbers like 2-4)
2. Change the angle to 60 degrees and observe the difference
3. Modify the rules to create your own variation
4. Try changing the distance parameter

Note: Be careful with high iteration numbers as they can create very complex patterns!

## Exercise 2: Plant Generation

Now let's create a more complex L-System that generates plant-like structures. This system uses brackets to create branches:

In [12]:
# Plant parameters
plant_axiom = "X"
plant_rules = {
    "X": "F+[[X]-X]-F[-FX]+X",
    "F": "FF"
}
plant_iterations = 4
plant_angle = 25

# Generate and draw
setup_turtle()
t.pensize(1)
plant_instructions = create_l_system(plant_iterations, plant_axiom, plant_rules)
draw_l_system(plant_instructions, plant_angle, 10)

## Challenge: Create a Fractal Tree

Now it's your turn to experiment! Below is a template for creating a fractal tree. Try modifying the parameters to create different tree shapes:

In [13]:
# Your fractal tree parameters
tree_axiom = "F"  # Start with a single trunk
tree_rules = {"F": "F[+F]F[-F]F"}  # Basic branching rule
tree_iterations = 3
tree_angle = 30

# Generate and draw
setup_turtle()
t.pensize(2)
tree_instructions = create_l_system(tree_iterations, tree_axiom, tree_rules)
draw_l_system(tree_instructions, tree_angle, 10)

#Generative Arts

In [28]:
def create_l_system(iterations, axiom, rules):
    """Generate L-System instructions based on axiom and rules."""
    result = axiom
    for _ in range(iterations):
        new_string = ""
        for char in result:
            new_string += rules.get(char, char)
        result = new_string
    return result

In [29]:
def setup_turtle(bg="white", start_mode="top", speed=13):
    """
    start_mode:
      - "top"    : start near top center, heading downward
      - "center" : start center, heading right
    """
    t.initializeTurtle()
    t.bgcolor(bg)
    t.hideturtle()
    t.speed(speed)
    t.penup()

    w, h = t.window_width(), t.window_height()

    if start_mode == "center":
        t.goto(w // 2, h // 2)
        t.setheading(0)
    else:
        t.goto(w // 2, h - 50)
        t.setheading(270)

    t.pendown()

In [46]:
def pick_color(progress=None, depth=None, mode="progress", palette_name="viridis_like"):
    """
    ColabTurtle supports named colors, so we map indices to a palette list.
    """
    palettes = {
        "viridis_like": [
            "midnightblue", "royalblue", "teal", "seagreen", "yellowgreen", "gold"
        ],
        "fairy": [
            "lavender", "thistle", "mistyrose",
            "powderblue", "honeydew", "lightcyan",
            "palegoldenrod", "peachpuff"
        ],
        "warm": [
            "deeppink", "hotpink", "orange","gold", "limegreen", "springgreen","deepskyblue"
        ],
        }
    palette = palettes.get(palette_name, palettes["viridis_like"])

    if mode == "depth" and depth is not None:
        idx = min(len(palette) - 1, depth)
        return palette[idx]
    else:
        if progress is None: progress = 0
        idx = int(progress * (len(palette) - 1))
        return palette[idx]


def thickness_from_depth(depth, base=2, decay=0.7):
    """Thicker trunk, thinner branches."""
    return max(1, int(base * (decay ** depth)))

In [47]:
def draw_l_system(instructions, angle, distance,
                  draw_symbols={"F"},
                  color_mode="progress",       # "progress" or "depth"
                  thickness_mode="constant",   # "constant" or "depth"
                  thickness_base=2,
                  thickness_decay=0.7,
                  palette_name="viridis_like"):
    """
    Draw the L-System using turtle graphics.

    Supports:
      - F (and any symbol in draw_symbols): draw forward
      - f: move forward no draw
      - +, -: turns
      - [, ]: branching stack
    """
    stack = []
    depth = 0
    total = len(instructions)

    for i, cmd in enumerate(instructions):
        progress = i / max(1, total - 1)

        # Draw-forward symbols
        if cmd in draw_symbols:
            # color
            if color_mode == "depth":
                t.color(pick_color(depth=depth, mode="depth", palette_name=palette_name))
            else:
                t.color(pick_color(progress=progress, mode="progress", palette_name=palette_name))

            # thickness
            if thickness_mode == "depth":
                t.pensize(thickness_from_depth(depth, base=thickness_base, decay=thickness_decay))
            else:
                t.pensize(thickness_base)

            t.forward(distance)

        elif cmd == "f":
            t.penup()
            t.forward(distance)
            t.pendown()

        elif cmd == "+":
            t.right(angle)

        elif cmd == "-":
            t.left(angle)

        elif cmd == "[":
            stack.append((t.position(), t.heading(), depth))
            depth += 1

        elif cmd == "]":
            pos, heading, depth = stack.pop()
            t.penup()
            t.goto(pos)
            t.setheading(heading)
            t.pendown()

In [48]:
SYSTEMS = {
    "Square Weave": dict(
        axiom="F-F-F-F",
        rules={"F": "F+F-F-F+F"},
        angle=90,
        distance=5,
        draw_symbols={"F"},
        start_mode="center"
    ),

    "Dragon Curve": dict(
        axiom="FX",
        rules={"X": "X+YF+", "Y": "-FX-Y"},
        angle=90,
        distance=4,
        draw_symbols={"F"},
        start_mode="center"
    ),

    "Hilbert Curve": dict(
        axiom="A",
        rules={"A": "+BF-AFA-FB+", "B": "-AF+BFB+FA-"},
        angle=90,
        distance=6,
        draw_symbols={"F"},
        start_mode="center"
    ),
}

In [ ]:
def run_5_outcomes():
    outcomes = [
        dict(preset="Square Weave", iterations=3, angle=90,
            color_mode="progress", thickness_mode="constant",
            thickness_base=3, palette_name="fairy"),

        dict(preset="Square Weave", iterations=4, angle=75,
            color_mode="depth", thickness_mode="depth",
            thickness_base=2, thickness_decay=0.8, palette_name="warm"),

        dict(preset="Hilbert Curve", iterations=4, angle=90,
            color_mode="progress", thickness_mode="constant",
            thickness_base=3, palette_name="viridis_like"),

        dict(preset="Dragon Curve", iterations=5, angle=80,
            color_mode="depth", thickness_mode="depth",
            thickness_base=5, thickness_decay=0.75, palette_name="fairy"),

        dict(preset="Dragon Curve", iterations=10, angle=100,
            color_mode="progress", thickness_mode="constant",
            thickness_base=1, palette_name="warm"),
    ]

    for cfg in outcomes:
        spec = SYSTEMS[cfg["preset"]]

        setup_turtle(bg="white", start_mode=spec.get("start_mode", "top"), speed=13)

        instr = create_l_system(cfg["iterations"], spec["axiom"], spec["rules"])

        draw_l_system(
            instr,
            angle=cfg.get("angle", spec["angle"]),
            distance=spec["distance"],
            draw_symbols=spec.get("draw_symbols", {"F"}),
            color_mode=cfg["color_mode"],
            thickness_mode=cfg["thickness_mode"],
            thickness_base=cfg["thickness_base"],
            thickness_decay=cfg.get("thickness_decay", 0.7),
            palette_name=cfg["palette_name"]
        )

run_5_outcomes()
